<img src="http://dask.readthedocs.io/en/latest/_images/dask_horizontal.svg"
     align="right"
     width="30%"
     alt="Dask logo\">


# Dask数据框

我们使用`dask.delayed`在CSV文件的目录上建立一个并行的数据帧计算，从而结束了第一章。 在本节中，我们使用`dask.dataframe`来自动构建类似的计算，用于常见的表格计算。 Dask数据框看起来和感觉都像Pandas数据框，但它们运行在与`dask.delayed`相同的基础设施上。

在这个笔记本中，我们使用了和以前一样的航空公司数据，但现在我们不写for-loops，而是让`dask.dataframe`为我们构造计算。 `dask.dataframe.read_csv`函数可以接受一个像`"data/nycflights/*.csv"`这样的globstring，并一次对我们所有的数据进行并行计算。

## 何时使用`dask.dataframe`？

Pandas对于能在内存中处理的表格数据集是非常优秀的工具。当你要分析的数据集大于你的机器内存时，Dask就变得很有用。我们正在使用的演示数据集只有大约200MB，所以你可以在合理的时间内下载它，但`dask.dataframe`将扩展到比内存大得多的数据集。

<img src="images/pandas_logo.png" align="right" width="28%">

`dask.dataframe`模块实现了一个阻塞的并行`DataFrame`对象，它模仿了Pandas`DataFrame`API的一个子集。一个Dask`DataFrame`是由许多内存中的pandas`DataFrame`组成，沿着索引分开。对Dask`DataFrame`的一个操作会触发对组成的pandas`DataFrame`的许多pandas操作，这种方式是注意潜在的并行性和内存限制。

**相关文档**

* [DataFrame documentation](https://docs.dask.org/en/latest/dataframe.html)
* [DataFrame screencast](https://youtu.be/AT2XtFehFSQ)
* [DataFrame API](https://docs.dask.org/en/latest/dataframe-api.html)
* [DataFrame examples](https://examples.dask.org/dataframe.html)
* [Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/)

**主要收获**

1.  Dask DataFrame应该是Pandas用户所熟悉的了
2.  数据框的分区对高效执行很重要。

## 构建数据

In [ ]:
%run prep.py -d flights

## Setup

In [ ]:
from dask.distributed import Client

client = Client(n_workers=4)

创建了人工数据。

In [ ]:
from prep import accounts_csvs
accounts_csvs()

import os
import dask
filename = os.path.join('data', 'accounts.*.csv')
filename

文件名包含一个 glob 模式 `*`，因此路径中与该模式匹配的所有文件都将被读入同一个 Dask DataFrame。

In [ ]:
import dask.dataframe as dd
df = dd.read_csv(filename)
df.head()

In [ ]:
# 加载计算行数
len(df)

这里发生了什么？
- Dask调查了输入路径，发现有三个匹配的文件。
- 为每个块智能地创建了一组作业--在这种情况下，每个原始CSV文件都有一个作业。
- 每个文件都被加载到一个pandas数据框中，并应用`len()`对其进行处理。
- 将小计合并，得出最后的总数。

### 真实数据

让我们用美国几年来的航班摘录来试试。这个数据是针对纽约市地区三个机场的航班的。

上市公司财务报表数据

In [ ]:
df = dd.read_csv(os.path.join('data', 'nycflights', '*.csv'),
                 parse_dates={'Date': [0, 1, 2]})

请注意，数据框对象的respresentation不包含任何数据--Dask只是做了足够的工作来读取第一个文件的开始，并推断出列名和dtypes。

In [ ]:
df

我们可以查看数据的开始和结束。

In [ ]:
df.head()

In [ ]:
df.tail()  # this fails

### 发生了什么？

与`pandas.read_csv`在推断数据类型之前读取整个文件不同，`dask.dataframe.read_csv`只读取文件开头的样本（如果使用glob，则读取第一个文件）。这些推断的数据类型会在读取所有分区时强制执行。

在这种情况下，样本中推断的数据类型是不正确的。前`n`行没有`CRSElapsedTime`的值（pandas推断为`float`），后来变成了字符串（`object`dtype）。请注意，Dask会给出一个关于不匹配的错误信息。当这种情况发生时，你有几个选择。

- 直接使用`dtype`关键字指定dtypes。这是推荐的解决方案，因为它是最不容易出错的（显式比隐式好），也是性能最好的。
- 增加`sample`关键字的大小（以字节为单位）。
- 使用 "assume_missing "使 "dask "假定推断为 "int "的列（不允许缺失值）实际上是floats（允许缺失值）。在我们的特殊情况下，这并不适用。

在我们的例子中，我们将使用第一个选项，直接指定违规列的`dtypes`。

In [ ]:
df = dd.read_csv(os.path.join('data', 'nycflights', '*.csv'),
                 parse_dates={'Date': [0, 1, 2]},
                 dtype={'TailNum': str,
                        'CRSElapsedTime': float,
                        'Cancelled': bool})

In [ ]:
df.tail()  # now works

## 用`dask.dataframe`进行计算

我们计算`DepDelay`列的最大值。如果只用pandas，我们会在每个文件上循环找到各个最大值，然后在所有的最大值上找到最后的最大值。

```python
maxes = []
for fn in filenames:
    df = pd.read_csv(fn)
    maxes.append(df.DepDelay.max())
    
final_max = max(maxes)
```

我们可以用`dask.delayed`来封装`d.read_csv`，这样它就可以并行运行。无论如何，我们还是要考虑循环、中间结果（每个文件一个）和最终的减少（中间最大值的`max`）。这只是真正的任务周围的噪音，pandas会用

```python
df = pd.read_csv(filename, dtype=dtype)
df.DepDelay.max()
```

`dask.dataframe`让我们可以编写类似于pandas的代码，对大于内存的数据集进行并行操作。

In [ ]:
%time df.DepDelay.max().compute()

这将为我们写入延迟计算，然后运行它。

一些需要注意的事情。

1.  和`dask.delayed`一样，我们需要在完成后调用`.compute()`。 在这之前，所有的东西都是懒惰的。
2.  Dask会尽快删除中间结果（比如每个文件的完整pandas数据框架）。
    - 这让我们可以处理比内存大的数据集。
    - 这意味着重复计算每次都要把所有的数据加载进来（再运行上面的代码，是比你预期的快还是慢？

与`Delayed`对象一样，你可以使用`.visualize`方法查看底层任务图。

In [ ]:
# notice the parallelism
df.DepDelay.max().visualize()

## 练习

在本节中，我们将进行一些`dask.dataframe`的计算。如果你对Pandas很熟悉，那么这些应该很熟悉。你将不得不考虑何时调用`compute`。

### 1.) 我们的数据集中有多少条记录？

如果你对pandas不熟悉，你会如何检查一个tuple的列表中有多少记录？

In [ ]:
# Your code here

In [ ]:
len(df)

### 2.) 总共乘坐了多少个未取消的航班？

如果是pandas，你会使用[布尔索引](https://pandas.pydata.org/pandas-docs/stable/indexing.html#boolean-indexing)。

In [ ]:
# Your code here

In [ ]:
len(df[~df.Cancelled])

### 3.) 每个机场总共有多少个未取消的航班？

*提示*: use [`df.groupby`](https://pandas.pydata.org/pandas-docs/stable/groupby.html).

In [ ]:
# Your code here

In [ ]:
df[~df.Cancelled].groupby('Origin').Origin.count().compute()

### 4.) 每个机场的平均起飞延误是多少？

注意，这和你在上一个笔记本中的计算结果是一样的（这种方法是快了还是慢了？

In [ ]:
# Your code here

In [ ]:
df.groupby("Origin").DepDelay.mean().compute()

### 5.) 一周中哪一天的平均出发延误最严重？

In [ ]:
# Your code here

In [ ]:
df.groupby("DayOfWeek").DepDelay.mean().compute()

## 分享中间成果

在计算上述所有操作时，我们有时会不止一次地进行相同的操作。对于大多数操作，`dask.dataframe`会对参数进行哈希，允许重复的计算被共享，并且只计算一次。

例如，让我们计算所有未取消航班的出发延误的平均值和标准差。由于dask操作是懒惰的，这些值还不是最终结果。它们只是得到结果所需的配方。

如果我们用两次调用计算来计算它们，就不会出现中间计算的共享。

In [ ]:
non_cancelled = df[~df.Cancelled]
mean_delay = non_cancelled.DepDelay.mean()
std_delay = non_cancelled.DepDelay.std()

In [ ]:
%%time

mean_delay_res = mean_delay.compute()
std_delay_res = std_delay.compute()

但让我们尝试将这两者传递给一个`compute`调用。

In [ ]:
%%time

mean_delay_res, std_delay_res = dask.compute(mean_delay, std_delay)

使用`dask.compute`大约需要1/2的时间。这是因为在调用`dask.compute`时，两个结果的任务图都被合并，使得共享操作只做一次而不是两次。特别是，使用`dask.compute`只做一次以下操作。

- 调用 "read_csv "和 "dask.compute"。
- 过滤器(`df[~df.Cancelled]`)
- 一些必要的还原("和"、"数")

要查看多个结果之间的合并任务图是什么样子的（以及共享的内容），可以使用`dask.visualize`函数（我们可能想使用`filename='graph.pdf'`将图保存到磁盘上，这样我们就可以更容易地放大）。

In [ ]:
dask.visualize(mean_delay, std_delay)

## 这与pandas相比，如何？

Pandas比`dask.dataframe`更成熟，功能更齐全。 如果你的数据适合放在内存中，那么你应该使用Pandas。 当你对不适合在内存中操作的数据集进行操作时，`dask.dataframe`模块给你提供了有限的`pandas`体验。

在本教程中，我们提供了一个由几个CSV文件组成的小数据集。 这个数据集在磁盘上是45MB，在内存中可扩展到约400MB。这个数据集足够小，你通常会使用Pandas。

我们选择这个大小是为了让练习快速完成。 Dask.dataframe只有在比这个大得多的问题上才真正有意义，当Pandas用可怕的

    MemoryError: ...

此外，分布式调度器允许相同的数据框架表达式在一个集群中执行。为了实现大规模的 "大数据 "处理，可以执行数据摄取函数，比如`read_csv`，数据存放在每个worker节点都可以访问的存储上（比如amazon的S3），由于大部分操作只从选择一些列开始，对数据进行转换和过滤，所以机器之间只需要进行相对少量的数据通信。

Dask.dataframe操作内部使用`pandas`操作。 一般来说，除了以下两种情况，它们的运行速度是差不多的。

1.  Dask引入了一点开销，每个任务大约1ms。 这通常可以忽略不计。
2.  当Pandas释放GIL时，`dask.dataframe`可以在一个进程内并行调用多个pandas操作，速度的提升与核心数成一定比例。对于不释放GIL的操作，需要多个进程才能获得同样的速度提升。

## Dask DataFrame 数据模型

在大多数情况下，Dask DataFrame感觉就像一个熊猫的DataFrame。
到目前为止，我们所看到的最大的区别是Dask的操作是懒惰的；它们会建立一个任务图，而不是立即执行（更多细节将在[Schedulers](05_distributed.ipynb)中介绍）。
这让Dask可以在核心之外并行地进行操作。

在[Dask数组](03_array.ipynb)中，我们看到一个`dask.array`是由许多NumPy数组组成，沿着一个或多个维度分块。
对于`dask.dataframe`来说也是如此：一个Dask DataFrame是由许多pandas DataFrames组成的。对于`dask.dataframe`来说，分块只沿着索引发生。

<img src="http://dask.pydata.org/en/latest/_images/dask-dataframe.svg" width="30%">

我们把每个分块称为*分区*，上/下界是*分部*。
Dask *可以*存储关于分区的信息。现在，当你写自定义函数应用于Dask DataFrames时，分区就会出现。

## 将 "CRSDepTime "转换为时间戳

该数据集存储的时间戳为`HHMM`，在`read_csv`中作为整数读入。

In [ ]:
crs_dep_time = df.CRSDepTime.head(10)
crs_dep_time

为了将这些转换为预定出发时间的时间戳，我们需要将这些整数转换为`pd.Timedelta`对象，然后将它们与`Date`列结合起来。

在pandas中，我们会使用`pd.to_timedelta`函数，并进行一些运算。

In [ ]:
import pandas as pd

# Get the first 10 dates to complement our `crs_dep_time`
date = df.Date.head(10)

# Get hours as an integer, convert to a timedelta
hours = crs_dep_time // 100
hours_timedelta = pd.to_timedelta(hours, unit='h')

# Get minutes as an integer, convert to a timedelta
minutes = crs_dep_time % 100
minutes_timedelta = pd.to_timedelta(minutes, unit='m')

# Apply the timedeltas to offset the dates by the departure time
departure_timestamp = date + hours_timedelta + minutes_timedelta
departure_timestamp

### 自定义代码和Dask数据框架

我们可以将 "pd.to_timedelta "换成 "dd.to_timedelta"，并在整个dask DataFrame上做同样的操作。但是，假设Dask没有实现`dd.to_timedelta`在Dask DataFrames上工作。那么你会怎么做呢？

`dask.dataframe`提供了一些方法来使应用自定义函数到Dask DataFrames更容易。

- [`map_partitions`](http://dask.pydata.org/en/latest/dataframe-api.html#dask.dataframe.DataFrame.map_partitions)
- [`map_overlap`](http://dask.pydata.org/en/latest/dataframe-api.html#dask.dataframe.DataFrame.map_overlap)
- [`reduction`](http://dask.pydata.org/en/latest/dataframe-api.html#dask.dataframe.DataFrame.reduction)

这里我们只讨论`map_partitions`，我们可以用它来自己实现`to_timedelta`。

In [ ]:
# Look at the docs for `map_partitions`

help(df.CRSDepTime.map_partitions)

基本的想法是将一个对DataFrame进行操作的函数应用到每个分区。
在本例中，我们将应用`pd.to_timedelta`。

In [ ]:
hours = df.CRSDepTime // 100
# hours_timedelta = pd.to_timedelta(hours, unit='h')
hours_timedelta = hours.map_partitions(pd.to_timedelta, unit='h')

minutes = df.CRSDepTime % 100
# minutes_timedelta = pd.to_timedelta(minutes, unit='m')
minutes_timedelta = minutes.map_partitions(pd.to_timedelta, unit='m')

departure_timestamp = df.Date + hours_timedelta + minutes_timedelta

In [ ]:
departure_timestamp

In [ ]:
departure_timestamp.head()

### 练习：重写上面的内容，只需调用 "map_partitions

这将比两次单独调用的效率略高，因为它减少了图中的任务数量。

In [ ]:
def compute_departure_timestamp(df):
    pass  # TODO: implement this

In [ ]:
departure_timestamp = df.map_partitions(compute_departure_timestamp)

departure_timestamp.head()

In [ ]:
def compute_departure_timestamp(df):
    hours = df.CRSDepTime // 100
    hours_timedelta = pd.to_timedelta(hours, unit='h')

    minutes = df.CRSDepTime % 100
    minutes_timedelta = pd.to_timedelta(minutes, unit='m')

    return df.Date + hours_timedelta + minutes_timedelta

departure_timestamp = df.map_partitions(compute_departure_timestamp)
departure_timestamp.head()

## 限制

### 哪些地方不能用？

Dask.dataframe只涵盖了Pandas API的一小部分，但使用得很好。
这种限制有两个原因。

1.  Pandas API是*大的
2.  有些操作确实很难并行完成（如排序）。

此外，一些重要的操作，如``set_index``可以工作，但比Pandas慢，因为它们包括大量的数据洗牌，可能会写到磁盘上。

## 了解更多


* [DataFrame documentation](https://docs.dask.org/en/latest/dataframe.html)
* [DataFrame screencast](https://youtu.be/AT2XtFehFSQ)
* [DataFrame API](https://docs.dask.org/en/latest/dataframe-api.html)
* [DataFrame examples](https://examples.dask.org/dataframe.html)
* [Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/)

In [ ]:
client.shutdown()